# 讀入套件和資料集

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

# Layers for FNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten

# Layers for CNN
from tensorflow.keras.layers import Conv2D, MaxPool2D, GlobalAveragePooling2D

from tensorflow.keras.optimizers import SGD, Adam

# For data preprocessing
from tensorflow.keras import datasets
from tensorflow.keras.utils import to_categorical

# 讀取資料集A並整理 (MNIST)

In [2]:
# Load MNIST
(x_train, y_train0), (x_test, y_test0) = datasets.mnist.load_data()

# Reshape size
x_train=x_train.reshape(-1, 28,28,1)
x_test=x_test.reshape(-1, 28,28,1)
# Normalize
x_train = x_train / x_train.max()
x_test = x_test / x_test.max()

# One-hot encoding
y_train = to_categorical(y_train0, 10)
y_test = to_categorical(y_test0, 10)

# 建立model_A資料集

### 修改部分=> 卷積層的filter數量下修為32, 64, 128，全連接層的神經元數量修至128。

In [17]:
CNN_layers = [Conv2D(32,(3,3), input_shape = (28, 28, 1), padding = 'same', activation = 'relu', name = 'Conv_1'),
              MaxPool2D(),
              Conv2D(64,(3,3), padding = 'same', activation='relu', name = 'Conv_2'),
              MaxPool2D(),
              Conv2D(128,(3,3), padding = 'same', activation = 'relu', name = 'Conv_3'),
              GlobalAveragePooling2D()]

FC_layers = [Dense(units = 128, activation = 'relu'),        
             Dense(units = 10, activation = 'softmax')]

In [18]:
model_A = Sequential(CNN_layers + FC_layers)

In [19]:
model_A.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv_1 (Conv2D)              (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
Conv_2 (Conv2D)              (None, 14, 14, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
Conv_3 (Conv2D)              (None, 7, 7, 128)         73856     
_________________________________________________________________
global_average_pooling2d_2 ( (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)              

# 訓練model_A

### 修改部分=> optimizer改為SGD, learning rate 改為0.02

In [20]:
model_A.compile(loss='categorical_crossentropy',
              optimizer = SGD(lr = 0.02),
              metrics = ['categorical_accuracy'])

In [21]:
model_A.fit(x_train, y_train, batch_size = 128, epochs = 5, validation_data = (x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 92s 2ms/sample - loss: 2.2906 - categorical_accuracy: 0.1873 - val_loss: 2.2733 - val_categorical_accuracy: 0.1760
Epoch 2/5
60000/60000 [==============================] - 102s 2ms/sample - loss: 2.2362 - categorical_accuracy: 0.2050 - val_loss: 2.1846 - val_categorical_accuracy: 0.2395
Epoch 3/5
60000/60000 [==============================] - 100s 2ms/sample - loss: 2.0586 - categorical_accuracy: 0.2798 - val_loss: 1.8556 - val_categorical_accuracy: 0.3585
Epoch 4/5
60000/60000 [==============================] - 133s 2ms/sample - loss: 1.6757 - categorical_accuracy: 0.4070 - val_loss: 1.3776 - val_categorical_accuracy: 0.5323
Epoch 5/5
60000/60000 [==============================] - 132s 2ms/sample - loss: 1.1791 - categorical_accuracy: 0.6054 - val_loss: 0.9900 - val_categorical_accuracy: 0.6524


# 查看model權重

In [22]:
train_score = model_A.evaluate(x_train, y_train)
test_score = model_A.evaluate(x_test, y_test)
print(f'Train Accuracy: {train_score[1]*100}')
print(f'Test Accuracy: {test_score[1]*100}')

10000/10000 [==============================] - 9s 864us/sample - loss: 0.9900 - categorical_accuracy: 0.6524
Train Accuracy: 65.07499814033508
Test Accuracy: 65.2400016784668


# 讀入B資料集並整理 (fashion_mnist)

In [24]:

(a_train, b_train), (a_test, b_test) = datasets.fashion_mnist.load_data()

a_train = a_train.reshape(60000, 28, 28, 1) / 255
a_test = a_test.reshape(10000, 28, 28, 1) / 255

b_train = to_categorical(b_train, 10)
b_test = to_categorical(b_test, 10)

In [32]:
FC_layers_CF = [Dense(units=256, activation='relu'),
                Dense(units=128, activation='relu'),
                Dense(units=10, activation='softmax')]

In [33]:
model_new = Sequential(CNN_layers + FC_layers_CF)
model_new.summary

<bound method Network.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x0000000BDD8A7F60>>

# Transfer learning (使用Frozen)

In [34]:
for layer in CNN_layers:
    layer.trainable = False

In [35]:
model_new.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv_1 (Conv2D)              (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
Conv_2 (Conv2D)              (None, 14, 14, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
Conv_3 (Conv2D)              (None, 7, 7, 128)         73856     
_________________________________________________________________
global_average_pooling2d_2 ( (None, 128)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 256)              

### 修改部分=> optimizer使用SGD learning = 0.02

In [36]:
model_new.compile(loss='categorical_crossentropy', 
                    optimizer = SGD(lr=0.02),
                    metrics=['categorical_accuracy'])

model_new.fit(a_train, b_train,batch_size=128, epochs=5,validation_data=(a_test, b_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 42s 705us/sample - loss: 1.3203 - categorical_accuracy: 0.5377 - val_loss: 1.0746 - val_categorical_accuracy: 0.5943
Epoch 2/5
60000/60000 [==============================] - 45s 747us/sample - loss: 0.9791 - categorical_accuracy: 0.6417 - val_loss: 0.9254 - val_categorical_accuracy: 0.6667
Epoch 3/5
60000/60000 [==============================] - 43s 718us/sample - loss: 0.8895 - categorical_accuracy: 0.6737 - val_loss: 0.8641 - val_categorical_accuracy: 0.6691
Epoch 4/5
60000/60000 [==============================] - 42s 700us/sample - loss: 0.8349 - categorical_accuracy: 0.6924 - val_loss: 0.8770 - val_categorical_accuracy: 0.6639
Epoch 5/5
60000/60000 [==============================] - 43s 709us/sample - loss: 0.8022 - categorical_accuracy: 0.7022 - val_loss: 0.8001 - val_categorical_accuracy: 0.6961


# 查看借來的神經網路權重是否有變化

In [38]:
train_score = model_A.evaluate(x_train, y_train)
test_score = model_A.evaluate(x_test, y_test)
print(f'Train Accuracy: {train_score[1]*100}')
print(f'Test Accuracy: {test_score[1]*100}')

10000/10000 [==============================] - 5s 532us/sample - loss: 0.9900 - categorical_accuracy: 0.6524
Train Accuracy: 65.07499814033508
Test Accuracy: 65.2400016784668


In [39]:
# 觀察: 因使用Frozen，故權重並無改變，不過用此進行遷移學習後的神經網路訓練成果微佳，正確率由原先60.54%上升到了70.22%